Moved this to a separate file (i.e. once OCR is done, I don't need to keep re-running it for tests)

In [2]:
%pip install -q \
    pandas \
    prisma

Note: you may need to restart the kernel to use updated packages.


In [2]:
#--schema=../receipts-app/prisma/schema.prisma
# !npm install -g prisma
# !prisma generate

# Setting up prisma python client was pretty hard. For now I copy the schema and .env file from the Next.js app folder
# then source .venv/bin/activate ... and run the prisma commands from there, including installing prisma, generate, and migrate
!cp ../receipts-app/prisma/dev.db ./prisma/dev.db
!cp ../receipts-app/prisma/schema.prisma ./prisma/schema.prisma
!perl -i -pe's/prisma-client-js/prisma-client-py/g' ./prisma/schema.prisma
!cp ../receipts-app/.env ./.env

In [3]:
import pandas as pd
import asyncio

In [4]:
# grab the CSVs with results from the OCR to save time
receipts_df = pd.read_csv('receipts.csv')
expenses_df = pd.read_csv('expenses.csv')

In [5]:
# # at first I tried with sqlite3 but I might as well use Prisma on both ends

# import sqlite3

# con = sqlite3.connect('../receipts-app/prisma/dev.db')
# df1 = pd.read_sql_query("SELECT * from UnitOfMeasure", con)

# print(df.head())

# con.close()

# # using sqlite3 directly

# import sqlite3

# con = sqlite3.connect('../receipts-app/prisma/dev.db')

# cur = con.cursor()

# # clear data first
# cur.execute("DELETE FROM UnitOfMeasure")

# # seed to sql lite my data UnitOfMeasure
# for unit in ['g', 'mL', 'Count']:
#     cur.execute("INSERT INTO UnitOfMeasure (name) VALUES (?)", (unit,))

# con.commit()
# con.close()


In [6]:
receipts_df

,box,text,confidence
0,"[[np.int32(176), np.int32(116)], [np.int32(738...",FRESH CO,0.839394
1,"[[np.int32(110), np.int32(281)], [np.int32(704...",CARLEI On PLACE LANSDOWNE FRESHCO,0.648093
2,"[[np.int32(220), np.int32(321)], [np.int32(281...",110,0.800745
3,"[[np.int32(291), np.int32(324)], [np.int32(591...",Larsclowne Averiue,0.535768
4,"[[np.int32(125), np.int32(361)], [np.int32(450...","CarIeton Place , ON",0.492920
...,...,...,...
169,"[[np.int32(482), np.int32(3748)], [np.int32(56...",iece,0.405077
170,"[[np.int32(573), np.int32(3744)], [np.int32(64...",ved,0.994361
171,"[[np.float64(76.02985749985467), np.float64(72...",(a (,0.231088
172,"[[np.float64(680.1135155856441), np.float64(32...",(rg:,0.344388


In [7]:
expenses_df

,filename,product,best_match,referenceitem,price
0,IMG_4558.jpg,"2"" Milk 1X21",Milk,Milk,65.59


In [20]:
# rather do it with prisma, so it's prisma on both sides
# wow, running primsa (npm) with python venv is a ride

from dotenv import load_dotenv
dotenv_path = '../receipts-app/.env'
load_dotenv(dotenv_path)

from prisma import Prisma

async def main():
    # Initialize the Prisma client
    db = Prisma()
    await db.connect()

    # see object attributes (for debug), note that the attributes are in all lowercase here
    # print(dir(db))
    
    # Static data
    unitOfMeasures = pd.DataFrame([
        {'name': 'g'},
        {'name': 'mL'},
        {'name': 'Count'}
    ])
    # # Dynamic data, sample for now
    referenceItems = pd.DataFrame([
        {
            'name': 'Item 1',
            'quantity': 1,
            'unitOfMeasure': 'Count',
            'price': 1.99,
            'pricePerWeight': 0.0009999,
            'referenceUrl': 'https://www.example.com/item1'
        },
        {
            'name': 'Item 2',
            'quantity': 1,
            'unitOfMeasure': 'Count',
            'price': 2.99,
            'pricePerWeight': 0.0009999,
            'referenceUrl': 'https://www.example.com/item1'
        },
        {
            'name': 'Item A',
            'quantity': 1,
            'unitOfMeasure': 'Count',
            'price': 3.99,
            'pricePerWeight': 0.0009999,
            'referenceUrl': 'https://www.example.com/item1'
        },
        {
            'name': 'Item B',
            'quantity': 1,
            'unitOfMeasure': 'Count',
            'price': 4.99,
            'pricePerWeight': 0.0009999,
            'referenceUrl': 'https://www.example.com/item1'
        }
    ])
    receipts = pd.DataFrame([
        {
            'filename': 'IMG_4549.jpg',
        },
        {
            'filename': 'IMG_4550.jpg',
        }
    ])
    receipttexts = pd.DataFrame([
        {'filename': 'IMG_4549.jpg', 'text': 'Item 1', 'boundingBox': '{"x":0,"y":0,"width":100,"height":20}', 'confidenceScore': 0.95},
        {'filename': 'IMG_4549.jpg', 'text': 'Item 2', 'boundingBox': '{"x":0,"y":20,"width":100,"height":20}', 'confidenceScore': 0.90},
        {'filename': 'IMG_4550.jpg', 'text': 'Item A', 'boundingBox': '{"x":0,"y":0,"width":100,"height":20}', 'confidenceScore': 0.93},
        {'filename': 'IMG_4550.jpg', 'text': 'Item B', 'boundingBox': '{"x":0,"y":20,"width":100,"height":20}', 'confidenceScore': 0.89}
    ])
    # expenses = pd.DataFrame([
    #     {
    #         'filename': 'IMG_4549.jpg',
    #         'referenceItem': 'Item 1',
    #         'bestMatch': 'Item 1',
    #         'price': 1.99
    #     },
    #     {
    #         'filename': 'IMG_4549.jpg',
    #         'referenceItem': 'Item 2',
    #         'bestMatch': 'Item 2',
    #         'price': 2.99
    #     },
    #     {
    #         'filename': 'IMG_4550.jpg',
    #         'referenceItem': 'Item A',
    #         'bestMatch': 'Item A',
    #         'price': 3.99
    #     },
    #     {
    #         'filename': 'IMG_4550.jpg',
    #         'referenceItem': 'Item B',
    #         'bestMatch': 'Item B',
    #         'price': 4.99
    #     }
    # ])

    # create units of measure
    # first clear db
    await db.referenceitem.delete_many(where={})
    await db.unitofmeasure.delete_many(where={})
    print('Db cleared')
    
    for unit in unitOfMeasures.to_dict(orient='records'):
        UoM = await db.unitofmeasure.create(data={'name': unit.get('name')})
        # add UoM.id to unitOfMeasures df once added to db
        unitOfMeasures.loc[unitOfMeasures['name'] == unit['name'], 'unitOfMeasureId'] = UoM.id
        # i could have manually specified foreign keys, but I like to use "remote" key values as a practice during imports
    unitOfMeasures['unitOfMeasureId'] = unitOfMeasures['unitOfMeasureId'].astype(int)
    # print(unitOfMeasures)

    # # change the referenceItems dataframe to use the UoM ids
    # # merge refrenceItems with unitOfMeasures to get the UoM ids keeping only the id column from unitOfMeasures
    # referenceItems = referenceItems.merge(unitOfMeasures, left_on='unitOfMeasure', right_on='name', suffixes=('', '_y'))
    # referenceItems = referenceItems.drop(columns=['name_y'])
    # referenceItems = referenceItems.drop(columns=['unitOfMeasure'])
    # print(referenceItems)

    # # for referenceItem in referenceItems.to_dict(orient='records'):
    # #     # find the UoM id for the unit of measure (next returns None if not found)
    # #     uom_id = next((uom.id for uom in uoms if uom.name == referenceItem['unitOfMeasure']), None)
    # #     if uom_id:
    # #         # Use the connect operation to link to an existing UnitOfMeasure
    # #         referenceItem['unitOfMeasure'] = {
    # #             'connect': {'id': uom_id}
    # #         }
    # # # delete column unitOfMeasurere in refrenceItems dataframe
    # # referenceItems.drop(columns=['unitOfMeasure'], inplace=True)

    # more elegant that joins or for loops
    unitOfMeasures_map = unitOfMeasures.set_index('name')['unitOfMeasureId'].to_dict()
    referenceItems['unitOfMeasureId'] = referenceItems['unitOfMeasure'].map(unitOfMeasures_map)
    print(referenceItems)

    # # create reference items
    # for referenceItem in referenceItems.to_dict(orient='records'):
    #     refrenceItem_db = await db.referenceitem.create(data=referenceItem)
    #     referenceItem['id'] = refrenceItem_db.id
    # referenceItems_db = await db.referenceitem.find_many()
    # print(referenceItems_db)

    # create receipts
    # ...
    # create receipt texts
    # ...

    # # test create expense
    # expense = await db.expense.create(
    #     data={
    #         'filename': 'IMG_4549.jpg',
    #         'priceEach': 1.99,
    #         'quantity': 1,
    #         ...

    # # create all the reference items
    # for referenceItem in referenceItems:
    #     await db.referenceItem.create(data=referenceItem)

    # Iterate through the receipts and create them along with their texts
    # for receipt_data in receipts:
    #     # Create the Receipt
    #     receipt = await db.receipt.create(
    #         data={
    #             'filename': receipt_data['filename'],
    #             'receiptText': {
    #                 'create': receipt_data['receiptTexts']
    #             }
    #         },
    #         include={
    #             'receiptText': True
    #         }
    #     )
    #     print(f'Receipt created: {receipt.id}, with texts: {[text.text for text in receipt.receiptText]}')

    # Disconnect the Prisma client
    await db.disconnect()

# Run the main function
# asyncio.run(main()) # not for jupyter
await main()

Db cleared
     name  quantity unitOfMeasure  price  pricePerWeight  \
0  Item 1         1         Count   1.99           0.001   
1  Item 2         1         Count   2.99           0.001   
2  Item A         1         Count   3.99           0.001   
3  Item B         1         Count   4.99           0.001   

                    referenceUrl  unitOfMeasureId  
0  https://www.example.com/item1               41  
1  https://www.example.com/item1               41  
2  https://www.example.com/item1               41  
3  https://www.example.com/item1               41  


In [5]:
# update Next.js app
!cp ./prisma/dev.db ../receipts-app/prisma/dev.db
!cp ./prisma/schema.prisma ../receipts-app/prisma/schema.prisma
!perl -i -pe's/prisma-client-py/prisma-client-js/g' ../receipts-app/prisma/schema.prisma
!cp ./.env ../receipts-app/.env